# Geekbench benchmark on Android

Geekbench4 is an app offering several benchmarks to run on android smartphones. The one used in this notebook is the '**CPU**' benchmark, which runs several workloads that follow the lines of what is commonly run by smartphones (AES, JPEG codec, FFT, and so on). The benchmark runs all the tests in '**Single-Core**' mode as well as in '**Multi-Core**' in order to compare the single-thread and multi-thread performances of the device.

**Do note that the benchmark will attempt to upload its results, which includes some hardware information**

In [ ]:
from conf import LisaLogging
LisaLogging.setup()

In [ ]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd

## Support Functions

This function helps us run our experiments:

In [ ]:
def experiment():
    
    # Configure governor
    target.cpufreq.set_all_governors('schedutil')
    
    # Get workload
    wload = Workload.getInstance(te, 'Geekbench')
    
    # Run Geekbench workload
    wload.run(te.res_dir, test_name='CPU', collect='ftrace')
        
    # Dump platform descriptor
    te.platform_dump(te.res_dir)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'android',
    "board"        : 'pixel',
    
    # Device
    "device"       : "HT67M0300128",
    
    # Android home
    "ANDROID_HOME" : "/home/vagrant/lisa/tools/android-sdk-linux/",

    # Folder where all the results will be collected
    "results_dir" : "Geekbench_example",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_overutilized",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "cpu_capacity",
            "cpu_frequency",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset'],
}

In [ ]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False)
target = te.target

## Workloads execution

This is done using the **experiment** helper function defined above which is configured to run a **Geekbench - CPU** experiment.

In [ ]:
# Initialize Workloads for this test environment
results = experiment()

## Results collection

Geekbench4 uses a baseline score of 4000, which is the benchmark score of an Intel Core i7-6600U. Higher scores are better, with double the score indicating double the performance. You can have a look at the results for several android phones here https://browser.primatelabs.com/android-benchmarks

This helper class will be used to parse the results

In [ ]:
class Geekbench(object):
    
    def __init__(self, filepath):
        with open(filepath) as fd:
            self.__json = json.loads(fd.read())
        
        self.__benchmarks = {}
        for section in self.__json["sections"]:
            self.__benchmarks[section["name"]] = section
            for workload in section["workloads"]:
                self.__benchmarks[section["name"]][workload["name"]] = workload
            
            
    def name(self):
        gov = ""
        build = ""
        for metric in self.__json["metrics"]:
            if metric["name"] == "Governor":
                gov = metric["value"]
            elif metric["name"] == "Build":
                build = metric["value"]

        return "[build]=\"{}\" [governor]=\"{}\"".format(build, gov)
    
    def __benchmarks_names(self):
        return [section["name"] for section in self.__json["sections"]]
    
    def __workloads_names(self):
        return [workload["name"] for workload in self.__benchmarks.values()[0]["workloads"]]
    
    def global_results(self):
        data = []
        for benchmark in self.__benchmarks_names():
            data.append(self.__benchmarks[benchmark]["score"])
        df = pd.DataFrame(data, index=self.__benchmarks_names(), columns=["Score"])
        return df
        
    def detailed_results(self):
        benchmark_fields = ["score", "runtime_mean", "rate_string"]
        dfs = {}

        benchmarks = self.__benchmarks_names()
        workloads = self.__workloads_names() 
        for benchmark in benchmarks:
            data = []
            idx = []
            for workload in workloads:
                wl_data = []
                for field in benchmark_fields:
                    wl_data.append(self.__benchmarks[benchmark][workload][field])            
                data.append(tuple(wl_data))
                idx.append(workload)  

            df = pd.DataFrame(data, index=idx, columns=benchmark_fields)
            dfs[benchmark] = df
            
        return dfs
        
    def compare_global(self, other):
        bench_types = self.__benchmarks_names()
        columns = ["New score", "Previous score", "Delta (%)"]
        data = []

        for bench_type in bench_types:
            other_score = other.__benchmarks[bench_type]["score"]
            my_score = self.__benchmarks[bench_type]["score"]
            delta = (1.0 * my_score - other_score) / other_score
            data.append((my_score, other_score, float("{:.2f}".format(delta * 100))))
            
        df = pd.DataFrame(data, index=bench_types, columns=columns)
        df.sort_values(by=columns[-1], inplace=True)
        return df
        
    def compare_detailed(self, other):
        bench_types = self.__benchmarks_names()
        columns = ["New score", "Previous score", "Delta (%)"]
        workloads = self.__workloads_names()
        dfs = {}
        
        for bench_type in bench_types:
            data = []
            idx = []
            other_bench = other.__benchmarks[bench_type]
            my_bench = self.__benchmarks[bench_type]
            
            for workload in workloads:
                other_score = other_bench[workload]["score"]
                my_score = my_bench[workload]["score"]
                delta = (1.0 * my_score - other_score) / other_score
                data.append((my_score, other_score, float("{:.2f}".format(delta * 100))))
                idx.append(workload)

            df = pd.DataFrame(data, index=idx, columns=columns)
            df.sort_values(by=columns[-1], inplace=True)
            dfs[bench_type] = df
            
        return dfs

In [ ]:
class TE():
    res_dir="/home/valsch01/Work/lisa/results/hikey_geekbench_eas"
    
te = TE()

In [ ]:
compare_dirs = [ "/home/valsch01/Work/lisa/ipynb/scratchpad/geekbench4/sched" ]

compare_geekbenches = []
for d in compare_dirs:
    files = [f for f in os.listdir(d) if f.endswith(".gb4")]
    for f in files:
        compare_geekbenches.append(Geekbench(d + "/" + f))

Here we parse the results from the benchmark run

In [ ]:
def display_bench_results(geekbench, detailed=False):
    print "===== Global results ====="
    display(geekbench.global_results())
    
    if not detailed:
        return
    
    print "===== Detailed results ====="
    for benchmark, df in geekbench.detailed_results().iteritems():
        print "----- {} benchmark -----".format(benchmark)
        display(df)
    
def compare_bench_results(geekbench, other, detailed=False):
    print "===== Global results ====="
    display(geekbench.compare_global(other))
        
    if not detailed:
        return
        
    print "===== Detailed results ====="
    dfs = geekbench.compare_detailed(other)
    for name, df in dfs.iteritems():
        print "----- {} -----".format(name)
        display(df)

In [ ]:
for f in os.listdir(te.res_dir):
    if f.endswith(".gb4"):
        geekbench = Geekbench(te.res_dir + "/" + f)
        
        print "Analysing geekbench {}".format(geekbench.name())
        display_bench_results(geekbench)
        
        for other in compare_geekbenches:
            print "Comparing to {}".format(other.name())
            compare_bench_results(geekbench, other)